In [18]:
!pip install gradio --quiet


In [19]:
from fastai.vision.all import *
import gradio as gr

In [20]:
path = untar_data(URLs.FOOD)

In [21]:
food_block = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224)
)

In [22]:
dls = food_block.dataloaders(path, bs=64)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(3)

epoch,train_loss,valid_loss,accuracy,time


In [9]:
learn.export("food_classifier.pkl")

In [10]:
learn = load_learner("food_classifier.pkl")

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [11]:
def classify_food(img):
    pred, pred_idx, probs = learn.predict(PILImage.create(img))
    return {learn.dls.vocab[i]: float(probs[i]) for i in range(len(probs))}


In [13]:
interface = gr.Interface(
    fn=classify_food,
    inputs=gr.Image(type="filepath", label="Yemek Fotoğrafı Yükle"),
    outputs=gr.Label(num_top_classes=3),
    title="🍽️ Yemek Tanıma Modeli",
    description="Bir yemek fotoğrafı yükle, model tahmin etsin!"
)
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://249f4c1cb91d965de9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
